In [170]:
import pandas as pd
df = pd.read_csv('dataProcessed.csv').dropna()
df.reset_index(drop=True, inplace=True)

In [171]:
MAXBODYSIZE = 300
MAXHEADSIZE = 100
EMBEDDINGDIM = 300

Stances = {'agree', 'disgree', 'discuss', 'unrelated'}

In [172]:
# lol bye memory :,( 
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec

model = KeyedVectors.load_word2vec_format('GensimVectors/GoogleNews-vectors-negative300.bin', binary=True)

MemoryError: 

In [157]:
df.head(5)

,Body ID,articleBody,Headline,Stance,headProcessed,bodyProcessed
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated,soldier shoot parliament lock down after gunfi...,a small meteorit crash into a wood area in nic...
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated,tourist dub spider man after spider burrow und...,a small meteorit crash into a wood area in nic...
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated,luke somer in fail rescu attempt in yemen,a small meteorit crash into a wood area in nic...
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated,break soldier shoot at war memori in ottawa,a small meteorit crash into a wood area in nic...
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated,giant catfish weigh stone catch in itali be th...,a small meteorit crash into a wood area in nic...


In [158]:
    
# unique bodies
Bodydf = df.groupby('Body ID').first()[['bodyProcessed']]
Bodydf.reset_index(inplace=True)
print(Bodydf.head(5))

# unique Heads
Headdf = df[['Body ID','headProcessed','Stance']]
print(Headdf.head(5))

   Body ID                                      bodyProcessed
0        0  a small meteorit crash into a wood area in nic...
1        4  last week we hint at what wa to come as ebola ...
2        5  newser wonder how long a quarter pounder with ...
3        6  post photo of a child onlin isi support announ...
4        7  at least suspect boko haram insurg be kill in ...
   Body ID                                      headProcessed     Stance
0        0  soldier shoot parliament lock down after gunfi...  unrelated
1        0  tourist dub spider man after spider burrow und...  unrelated
2        0          luke somer in fail rescu attempt in yemen  unrelated
3        0        break soldier shoot at war memori in ottawa  unrelated
4        0  giant catfish weigh stone catch in itali be th...  unrelated


In [159]:
Headdf.head(5)

,Body ID,headProcessed,Stance
0,0,soldier shoot parliament lock down after gunfi...,unrelated
1,0,tourist dub spider man after spider burrow und...,unrelated
2,0,luke somer in fail rescu attempt in yemen,unrelated
3,0,break soldier shoot at war memori in ottawa,unrelated
4,0,giant catfish weigh stone catch in itali be th...,unrelated


In [160]:
''' set maximum dimension '''
colIndex = Bodydf.columns.get_loc("bodyProcessed")
for index,row in Bodydf.iterrows():
    Bodydf.iat[index, colIndex] = " ".join([word for word in row['bodyProcessed'].split(' ') if word in model][:MAXBODYSIZE])
Bodydf.to_csv('TrainBodyEmbed.csv',index=False)
    
colIndex = Headdf.columns.get_loc("headProcessed")
for index,row in Headdf.iterrows():
    Headdf.iat[index, colIndex] = " ".join([word for word in row['headProcessed'].split(' ') if word in model][:MAXHEADSIZE])
Headdf.to_csv('TrainHeadEmbed.csv',index=False)


TypeError: argument of type 'Model' is not iterable

In [161]:
totalText = []

In [162]:
def totalWordsForEmbedding(dfHead, dfBody):
    global totalText
    for index, row in dfHead.iterrows():
        totalText.append(row['headProcessed'].split(' '))
    for index, row in dfBody.iterrows():
        totalText.append(row['bodyProcessed'].split(' '))

In [163]:
totalWordsForEmbedding(Headdf, Bodydf)

In [164]:
''' CONDENSED FOR TEST '''

df = pd.read_csv('testProcessed.csv').dropna()
df.reset_index(drop=True, inplace=True)
Bodydf = df.groupby('Body ID').first()[['bodyProcessed']]
Bodydf.reset_index(inplace=True)
print(Bodydf.head(5))

# unique Heads
Headdf = df[['Body ID','headProcessed']]
print(Headdf.head(5))
''' set maximum dimension '''
colIndex = Bodydf.columns.get_loc("bodyProcessed")
for index,row in Bodydf.iterrows():
    Bodydf.iat[index, colIndex] = " ".join([word for word in row['bodyProcessed'].split(' ') if word in model][:MAXBODYSIZE])
Bodydf.to_csv('TestBodyEmbed.csv',index=False)
    
colIndex = Headdf.columns.get_loc("headProcessed")
for index,row in Headdf.iterrows():
    Headdf.iat[index, colIndex] = " ".join([word for word in row['headProcessed'].split(' ') if word in model][:MAXHEADSIZE])
Headdf.to_csv('TestHeadEmbed.csv',index=False)

totalWordsForEmbedding(Headdf, Bodydf)

   Body ID                                      bodyProcessed
0        1  ha deni isra report state that he offer to ext...
1        2  a bereav afghan mother take reveng on the tali...
2        3  cnbc be report tesla ha choose nevada as the s...
3       12  a version of the iphon be say to be in develop...
4       19  gr editor s note there be no report in the wes...
   Body ID                                      headProcessed
0        1        appl instal safe to protect gold watch edit
1        1  deni claim he will give sinai land to palestinian
2        1       appl to keep gold watch edit in special safe
3        1  appl store to keep gold edit appl watch in cus...
4        1  south korean woman hair by robot vacuum cleane...


TypeError: argument of type 'Model' is not iterable

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

In [165]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(totalText)
wordIndexs = tokenizer.word_index
vocabSize = tokenizer.word_counts
print(len(wordIndexs), len(vocabSize), tokenizer.document_count)

14479 14479 51653


In [166]:
wordIndexsdf = pd.DataFrame.from_dict(wordIndexs, orient='index')
wordIndexsdf.to_csv('wordIndexs.csv',index=False)
wordIndexsdf.head(10)

,0
the,1
to,2
in,3
of,4
be,5
a,6
and,7
on,8
that,9
for,10


In [167]:
embeddingVector = {}
for word, index in wordIndexs.items():
    if word != '':
        embeddingVector[index] = model[word]
embeddingdf = pd.DataFrame.from_dict(embeddingVector, orient='index')
embeddingdf.head(10)

TypeError: 'Model' object is not subscriptable

In [17]:
embeddingdf.to_csv('embeddingVectors.csv',index=False)

In [18]:
trainBodyDf = pd.read_csv('TrainBodyEmbed.csv')
trainHeadDf = pd.read_csv('TrainHeadEmbed.csv')

In [54]:
trainBodyDf.head(5)

,Body ID,bodyProcessed
0,0,small crash into wood area in capit overnight ...
1,4,last week we hint at what wa come as ebola fea...
2,5,newser wonder how long quarter pounder with ca...
3,6,post photo child onlin isi support announc tha...
4,7,at least suspect haram be kill in clash betwee...


In [141]:
trainHeadDf.head(5)

,Body ID,headProcessed,Stance
0,0,soldier shoot parliament lock down after erupt...,unrelated
1,0,tourist dub spider man after spider burrow und...,unrelated
2,0,luke in fail attempt in yemen,unrelated
3,0,break soldier shoot at war in ottawa,unrelated
4,0,giant catfish weigh stone catch in be think be...,unrelated


In [153]:
def CreateTrainingData(bodydf, headdf):
    heads = []
    bodies = []
    stances = []
    stancesLookup = {'unrelated': 0 , 'agree':1, 'disagree':2, 'discuss':3}
    for index, row in headdf.head(10).iterrows():
        heads.append(row['headProcessed'].split(" "))
        bodies.append(bodydf.loc[bodydf['Body ID'] == row['Body ID']]['bodyProcessed'][0].split(" "))
        stances.append(stancesLookup[row['Stance'].strip()])
    heads = tokenizer.texts_to_sequences(heads)
    bodies = tokenizer.texts_to_sequences(bodies)
#     heads = pad_sequences(heads,maxlen = MAXHEADSIZE,padding = 'post')
#     bodies = pad_sequences(bodies,maxlen = MAXBODYSIZE,padding = 'post')
    stances = to_categorical(stances, num_classes=4)
    return heads,bodies,stances
heads,bodies,stances = CreateTrainingData(trainBodyDf, trainHeadDf)

In [154]:
len(heads)

10

In [113]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Input, Lambda
from keras.layers.wrappers import Bidirectional
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD, Adagrad
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D, AveragePooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM, GRU
from keras.layers import concatenate,dot
from keras.preprocessing import sequence
import keras.backend as K
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
import pydotplus
from keras.utils.vis_utils import model_to_dot
import keras
keras.utils.vis_utils.pydot = pydot


In [87]:
import numpy as np
embedMatrix = np.zeros((len(tokenizer.word_counts)+1 , EMBEDDINGDIM))
for word, index in wordIndexs.items():
    if word != '':
        embedMatrix[index] = embeddingVector[tokenizer.texts_to_sequences([word])[0][0]]

In [117]:


InputHead = Input(shape=(MAXBODYSIZE,), dtype='int32', name='InputHead')
InputBody = Input(shape=(MAXHEADSIZE,), dtype='int32', name='InputBody')
Embeddings = Embedding(len(wordIndexs) + 1, EMBEDDINGDIM, weights=[embedMatrix],trainable=False)
EmbedHead = Embeddings(InputHead)
EmbedBody = Embeddings(InputBody)



LSTMHead = Bidirectional(LSTM(64,dropout=0.2, recurrent_dropout=0.2, name='LSTMHead'))(EmbedHead)
LSTMBody = Bidirectional(LSTM(64,dropout=0.2, recurrent_dropout=0.2, name='LSTMBody'))(EmbedBody)

DotLayer = dot([LSTMHead,LSTMBody],axes = 1, normalize=True)

Concat = concatenate([LSTMHead,LSTMBody,DotLayer])

DenseLayer = Dense(100,activation='relu')(Concat)
DenseLayer = Dropout(0.3)(DenseLayer)
DenseLayer = Dense(4,activation='softmax')(DenseLayer)
LSTMNetwork = Model(inputs=[InputHead,InputBody], outputs=[DenseLayer])
LSTMNetwork.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])
print(LSTMNetwork.summary())


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
InputHead (InputLayer)          (None, 300)          0                                            
__________________________________________________________________________________________________
InputBody (InputLayer)          (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_19 (Embedding)        multiple             1625100     InputHead[0][0]                  
                                                                 InputBody[0][0]                  
__________________________________________________________________________________________________
bidirectional_35 (Bidirectional (None, 128)          186880      embedding_19[0][0]               
__________

In [ ]:
''' CANT GET TO WORK? shows network graph, currently a keras bug '''
plot_model(LSTMNetwork, to_file='LSTMNetworkMap.png')
SVG(model_to_dot(LSTMNetwork).create(prog='dot', format='svg'))

'''

If you are getting errors here for pydot install graphviz & pydot & pydotplus
GraphViz needs to be installed using apt-get / brew
otherwise comment out sainvg\

'''

In [ ]:
print(len)

In [137]:
len(stances)
TrainHeadSplit, TrainBodySplit, TrainStanceSplit = bodies[:splitNum], heads[:splitNum], stances[:splitNum]
TestHeadSplit, TestBodySplit, TestStanceSplit = bodies[splitNum:], heads[splitNum:], stances[splitNum:]



for i in range(1):
    print(len(bodies[0]))
    model.fit([TrainBodySplit,TrainHeadSplit],[TrainStanceSplit], validation_data = ([TestBodySplit,TestHeadSplit],[TestStanceSplit]),epochs=4, batch_size=128,verbose = True)

300


ValueError: Error when checking input: expected head_input to have shape (300,) but got array with shape (100,)